<a href="https://colab.research.google.com/github/annkvth/kompleteNetze/blob/main/GetDataTrafficTrino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code to retrieve data from the opensky database

See example from the documentation: https://traffic-viz.github.io/gallery/kaliningrad.html

Note: there's no need to store username and password in a config file - the jupyter output will feature a link for trino authentication in a separate browser window

Installing the traffic python package if necessary


In [98]:
!pip install traffic



Include libraries

In [99]:
from datetime import datetime, timedelta
import pandas as pd
#from cartes.crs import Lambert93  # type: ignore

Always use python engine for dataframe queries, see
https://stackoverflow.com/questions/67063643/is-there-a-way-to-force-pandas-dataframe-query-to-use-python-as-default-engine

In [100]:
from functools import partialmethod

pd.DataFrame.query = partialmethod(pd.DataFrame.query, engine="python")



... and the traffic library used to hadle the opensky queries

In [101]:
import traffic
traffic.config_file

PosixPath('/root/.config/traffic/traffic.conf')

In [102]:
from traffic.data import opensky
#from traffic.core import Flight
#from traffic.data import eurofirs


Define list of airports we are interested in: based on https://de.wikipedia.org/wiki/Liste_der_Verkehrsflugh%C3%A4fen_in_Deutschland

In [103]:
icaolist=["EDBH", "EDDB", "EDVE", "EDDW", "ETMN", "EDLW", "EDDC", "EDDL", "EDDE", "EDDF", "EDFH", "EDNY", "EDDH", "EDDV", "EDAH", "ETSI", "EDSB", "EDVK",
           "EDDK", "EDTL", "EDDP", "EDHL", "EDBC", "EDJA", "EDDM", "EDDG", "EDBN", "EDLV", "EDDN", "EDMO", "EDLP", "ETNL", "EDDR", "EDGS", "EDDS", "EDXW"]
# note: it was necessary to split the list in two parts, having one giant list was not supported by the database query
print(icaolist)

['EDBH', 'EDDB', 'EDVE', 'EDDW', 'ETMN', 'EDLW', 'EDDC', 'EDDL', 'EDDE', 'EDDF', 'EDFH', 'EDNY', 'EDDH', 'EDDV', 'EDAH', 'ETSI', 'EDSB', 'EDVK', 'EDDK', 'EDTL', 'EDDP', 'EDHL', 'EDBC', 'EDJA', 'EDDM', 'EDDG', 'EDBN', 'EDLV', 'EDDN', 'EDMO', 'EDLP', 'ETNL', 'EDDR', 'EDGS', 'EDDS', 'EDXW']


First test: let's get the flightlist of all flights starting at any airport from the list

 The syntax is:
 OpenSky.flightlist(start, stop=None, *args, departure_airport=None, arrival_airport=None, airport=None, callsign=None, icao24=None, cached=True, compress=False, limit=None, **kwargs)

In [136]:
# time range to retrieve (note: it seems stopdate is exclusive)
startdate="2019-12-01"
stopdate="2020-01-01"

In [137]:
pdDeparture = opensky.flightlist(
    startdate,
    stopdate,
    departure_airport=icaolist,
)


Open the following URL in browser for the external authentication:
https://trino.opensky-network.org/oauth2/token/initiate/d5faade412f4a6a9715deae3b16b63005e27f0af9d976f5d8e716d48bd634bfe


FINISHED: : 100% [00:22, 4.51%/s]
DOWNLOAD: 64.5klines [00:05, 11.2klines/s]


In [138]:
print(f'retrieved {len(pdDeparture)} flights departing at any of the selected airports')


retrieved 64491 flights departing at any of the selected airports


In [139]:
pdArrival = opensky.flightlist(
    startdate,
    stopdate,
    arrival_airport=icaolist,
)


Open the following URL in browser for the external authentication:
https://trino.opensky-network.org/oauth2/token/initiate/ddd22ab27d78cec00da6099a6de04c025a3ae4ecbdbb474d66cabcec7edab48d


FINISHED: : 100% [00:23, 4.33%/s]
DOWNLOAD: 65.2klines [00:05, 11.0klines/s]


In [140]:
print(f'retrieved {len(pdArrival)} flights arriving at any of the selected airports')

retrieved 65178 flights arriving at any of the selected airports


Now combine the two, while dropping duplicate rows (i.e. flights start went from one airport to another on the list should only be listed once).

Output the (beginning and the )end of the retrieved dataframe

In [153]:
pdAll = pd.concat([pdDeparture, pdArrival]).drop_duplicates().reset_index(drop=True)

In [154]:
print(f'retrieved {len(pdAll)} flights departing or arriving at any of the selected airports')

retrieved 115935 flights departing or arriving at any of the selected airports


Just a quick look how the information inside the data frame looks like

In [155]:
pdAll.tail()

,icao24,firstseen,departure,lastseen,arrival,callsign,day
115930,4690ec,2019-12-04 07:05:32+00:00,<NA>,2019-12-04 08:18:27+00:00,EDDM,AEE5CM,2019-12-04 00:00:00+00:00
115931,471eaa,2019-12-04 08:57:37+00:00,<NA>,2019-12-04 10:06:24+00:00,EDDB,WZZ6769,2019-12-04 00:00:00+00:00
115932,471eaa,2019-12-04 04:27:57+00:00,<NA>,2019-12-04 05:40:53+00:00,EDDB,WZZ435,2019-12-04 00:00:00+00:00
115933,3c648a,2019-12-04 10:01:10+00:00,EDDT,2019-12-04 10:59:19+00:00,EDDF,DLH039,2019-12-04 00:00:00+00:00
115934,3e2535,2019-12-04 16:22:03+00:00,<NA>,2019-12-04 17:41:04+00:00,EDDS,EFD2D,2019-12-04 00:00:00+00:00


Sort the data frame by departure time

In [160]:
pdSorted = pdAll.sort_values(by=['day'])
pdSorted.tail()


,icao24,firstseen,departure,lastseen,arrival,callsign,day
110798,407536,2019-12-31 17:37:45+00:00,EGLL,2019-12-31 18:28:19+00:00,EDDL,BAW932,2019-12-31 00:00:00+00:00
110799,3c65c1,2019-12-31 14:07:52+00:00,EDDT,2019-12-31 15:04:45+00:00,EDDF,DLH5CK,2019-12-31 00:00:00+00:00
110800,3c66e2,2019-12-31 12:01:37+00:00,<NA>,2019-12-31 13:28:13+00:00,EDDM,DLH5TN,2019-12-31 00:00:00+00:00
110801,4b1a65,2019-12-31 17:16:19+00:00,LSGG,2019-12-31 18:40:24+00:00,EDDH,EZS85BF,2019-12-31 00:00:00+00:00
110802,4bcde3,2019-12-31 08:28:17+00:00,<NA>,2019-12-31 11:25:27+00:00,EDDS,SXS4WK,2019-12-31 00:00:00+00:00


Mount google drive to save the resulting csv file

In [145]:
from google.colab import drive
drive.mount('/content/drive')

import os

# where to save the data (and making shure that the directory exist)
folder_path = "/content/drive/My Drive/2024KomplexeNetze/Daten"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Last step: save the retrieved data into csv, with a name defined by start and stop date.

In [158]:
outputfile=startdate+"_"+stopdate+".csv"
file_path = os.path.join(folder_path, outputfile)
pdSorted.to_csv(file_path, index=False)  # ((saving without the DataFrame index))

In [159]:
print(f'Output file has been generated in {file_path}')

Output file has been generated in /content/drive/My Drive/2024KomplexeNetze/Daten/2019-12-01_2020-01-01.csv


--------------------------------------
Note: if we want, we could also get some airport info from provided by traffic, like geo coordinates, city, name.

In [148]:
from traffic.data import airports
# the command below is also getting Hamburg, USA, and hospital helipads in HH, ...
#airports.search("Hamburg")


In [149]:
airports.query('icao=="EDDB" and country=="Germany"')

,name,iata,icao,latitude,longitude,country,altitude,type,municipality
22735,Berlin Brandenburg Airport,BER,EDDB,52.362247,13.500672,Germany,157.0,large_airport,Berlin


I thought about adding this information to the output file - but that would just increase the file size a lot, we can just add that info at the end of the analysis for making nice plots, and work with icao until then.

TO DO: check the list of airports in the published covid-19 dataset - are they the same airports?
